In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None
%config Completer.use_jedi = False # Autocomplete works faster

In [2]:
# Importing data
df = pd.read_csv('./data/occurrence.txt', sep='\t')

/home/davidgp/anaconda3/envs/malifo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (198,237) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df.dtypes

gbifID                  int64
abstract              float64
accessRights          float64
accrualMethod         float64
accrualPeriodicity    float64
                       ...   
level1Name             object
level2Gid              object
level2Name             object
level3Gid              object
level3Name             object
Length: 249, dtype: object

In [11]:
df.columns

Index(['gbifID', 'abstract', 'accessRights', 'accrualMethod',
       'accrualPeriodicity', 'accrualPolicy', 'alternative', 'audience',
       'available', 'bibliographicCitation',
       ...
       'recordedByID', 'identifiedByID', 'level0Gid', 'level0Name',
       'level1Gid', 'level1Name', 'level2Gid', 'level2Name', 'level3Gid',
       'level3Name'],
      dtype='object', length=249)

In [21]:
# Cheking if all values are NaN
df.columns[df.isna().all()]

Index(['abstract', 'accessRights', 'accrualMethod', 'accrualPeriodicity',
       'accrualPolicy', 'alternative', 'audience', 'available',
       'bibliographicCitation', 'conformsTo',
       ...
       'nomenclaturalStatus', 'taxonRemarks', 'distanceAboveSurface',
       'distanceAboveSurfaceAccuracy', 'mediaType', 'subgenusKey',
       'typifiedName', 'relativeOrganismQuantity', 'recordedByID',
       'identifiedByID'],
      dtype='object', length=167)

In [22]:
# Deleting columns where all values are NaN. There some columns with no data because the file was made
# acording to the Darwin Core standard: https://dwc.tdwg.org/terms/
df.dropna(how='all', axis=1, inplace=True)

In [23]:
df.columns

Index(['gbifID', 'identifier', 'license', 'modified', 'references', 'rights',
       'rightsHolder', 'type', 'datasetID', 'institutionCode',
       'collectionCode', 'datasetName', 'ownerInstitutionCode',
       'basisOfRecord', 'occurrenceID', 'catalogNumber', 'recordedBy',
       'occurrenceStatus', 'occurrenceRemarks', 'eventDate', 'year', 'month',
       'day', 'samplingProtocol', 'waterBody', 'countryCode', 'locality',
       'verbatimLocality', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'georeferenceSources', 'identifiedBy',
       'dateIdentified', 'taxonID', 'scientificName', 'nameAccordingTo',
       'kingdom', 'phylum', 'class', 'order', 'family', 'genus',
       'specificEpithet', 'infraspecificEpithet', 'taxonRank',
       'taxonomicStatus', 'datasetKey', 'publishingCountry', 'lastInterpreted',
       'elevation', 'elevationAccuracy', 'depth', 'depthAccuracy', 'issue',
       'hasCoordinate', 'hasGeospatialIssues', 'taxonKey', 'acceptedTa

In [49]:
# Checking if there are duplicates
df.drop_duplicates().shape, df.shape

((38324, 82), (38324, 82))

In [56]:
# Let's check issues in the data. Issue column contains general information and hasGeospatialIssues column
# has geospatial problems
df['issue'].value_counts()

COORDINATE_REPROJECTION_SUSPICIOUS;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                     36277
COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;COUNTRY_INVALID;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE          1609
COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                           120
ZERO_COORDINATE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                                          115
COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH_FUZZY;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                      63
COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH_NONE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                       44
ZERO_COORDINATE;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                               42
COORDINATE_REPROJECTION_SUSPICIOUS

In [74]:
# It seems that all rows have the issues
# INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE  
# and COORDINATE_REPROJECTION_SUSPICIOUS or ZERO_COORDINATE
df[((df['issue'].str.contains('COORDINATE_REPROJECTION_SUSPICIOUS')) 
    | (df['issue'].str.contains('ZERO_COORDINATE'))) & 
  (df['issue'].str.contains('INSTITUTION_MATCH_NONE')) &
  (df['issue'].str.contains('COLLECTION_MATCH_NONE'))
  ].shape, df.shape

((38324, 82), (38324, 82))

In [75]:
# Let's check some issues, like COUNTRY_INVALID, COUNTRY_COORDINATE_MISMATCH, ZERO_COORDINATE,
# TAXON_MATCH_FUZZY, TAXON_MATCH_NONE, TAXON_MATCH_HIGHERRANK, COUNTRY_DERIVED_FROM_COORDINATES

df[df['issue'].str.contains('ZERO_COORDINATE')][['decimalLatitude','decimalLongitude']].value_counts()

decimalLatitude  decimalLongitude
0.0              0.0                 158
dtype: int64

In [58]:
df[df['issue'].str.contains('COUNTRY_INVALID')]['countryCode'].value_counts()

ZZ    1609
BQ       2
SA       1
Name: countryCode, dtype: int64

In [57]:
df[df['issue'].str.contains('COUNTRY_COORDINATE_MISMATCH')]['countryCode'].value_counts()

ZZ    1609
NL      59
SX      22
ZA      18
EG      15
MV      11
AI       5
CU       5
BS       4
AU       4
VI       3
NO       3
VG       2
TR       2
FJ       2
TH       2
EC       1
US       1
DK       1
UM       1
HN       1
Name: countryCode, dtype: int64

In [77]:
df[df['issue'].str.contains('TAXON_MATCH_FUZZY')]

,gbifID,identifier,license,modified,references,rights,rightsHolder,type,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,recordedBy,occurrenceStatus,occurrenceRemarks,eventDate,year,month,day,samplingProtocol,waterBody,countryCode,locality,verbatimLocality,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferenceSources,identifiedBy,dateIdentified,taxonID,scientificName,nameAccordingTo,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,issue,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,speciesKey,species,genericName,acceptedScientificName,verbatimScientificName,protocol,lastParsed,lastCrawled,repatriated,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
1742,1946390640,diveboard:412051_601063_0,CC_BY_NC_4_0,2018-10-11T11:04:49Z,https://www.diveboard.com/coggy41/D1fEDiL,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:412051_601063_0,diveboard:412051_601063_0,Coggy41,PRESENT,Non validated citizen science record,2018-10-08T11:27:00,2018,10,8,citizen science scuba dive observation,Atlantic Ocean,ES,Playa Chica,"Playa Chica, Puerto del Carmen, Lanzarote",28.91880,-13.67290,100.0,Google Maps,Coggy41,2018-10-08T00:00:00,http://eol.org/pages/601063,"Luidia sarsii Düben & Koren, 1844",EOL,Animalia,Echinodermata,Asteroidea,Paxillosida,Luidiidae,Luidia,sarsii,NaN,SPECIES,ACCEPTED,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:12.960Z,0.0,0.0,11.00,11.00,COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH...,True,False,2270638,2270638.0,1,50.0,214.0,808.0,9474.0,2270561.0,2270638.0,Luidia sarsii,Luidia,"Luidia sarsii Düben & Koren, 1844",Luidia sarsi,DWC_ARCHIVE,2020-12-15T21:04:12.960Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3030,1946389352,diveboard:381936_1005866_0,CC_BY_NC_4_0,2017-11-21T17:06:33Z,https://www.diveboard.com/ubiktrash/D2j3qRV,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:381936_1005866_0,diveboard:381936_1005866_0,Ubiktrash@Gmail.Com,PRESENT,Non validated citizen science record,2017-11-08T04:06:00,2017,11,8,citizen science scuba dive observation,Lanzarote,ES,"Lanzarote, Puerto Del Carmen, Blue Hole","Lanzarote, Puerto Del Carmen, Blue Hole, Lanza...",28.91890,-13.67030,100.0,Google Maps,Ubiktrash@Gmail.Com,2017-11-08T00:00:00,http://eol.org/pages/1005866,"Muggiaea kochii (Will, 1844)",EOL,Animalia,Cnidaria,Hydrozoa,Siphonophorae,Diphyidae,Muggiaea,kochii,NaN,SPECIES,ACCEPTED,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:15.140Z,8.0,0.0,16.00,16.00,COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH...,True,False,7939922,7939922.0,1,43.0,205.0,6180736.0,8188.0,2264794.0,7939922.0,Muggiaea kochii,Muggiaea,"Muggiaea kochii (Will, 1844)","Muggiaea kochi (Will, 1844)",DWC_ARCHIVE,2020-12-15T21:04:15.140Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3777,1585878508,diveboard:366778_601063_0,CC_BY_NC_4_0,2018-07-24T18:27:21Z,NaN,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:366778_601063_0,diveboard:366778_601063_0,Anonymous,PRESENT,Non validated citizen science record,2017-07-08T11:33:00,2017,7,8,citizen science scuba dive observation,Irish Sea,IE,The Outfall (The Lady's Stairs),"The Outfall (The Lady's Stairs), Dun Laoghaire",53.28830,-6.11159,100.0,G

In [78]:
df[df['issue'].str.contains('TAXON_MATCH_NONE')]

,gbifID,identifier,license,modified,references,rights,rightsHolder,type,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,recordedBy,occurrenceStatus,occurrenceRemarks,eventDate,year,month,day,samplingProtocol,waterBody,countryCode,locality,verbatimLocality,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferenceSources,identifiedBy,dateIdentified,taxonID,scientificName,nameAccordingTo,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,issue,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,speciesKey,species,genericName,acceptedScientificName,verbatimScientificName,protocol,lastParsed,lastCrawled,repatriated,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
1315,2247810902,diveboard:426432_1301584_0,CC_BY_NC_4_0,2019-05-11T13:12:27Z,https://www.diveboard.com/greencr83/D189Gw7,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:426432_1301584_0,diveboard:426432_1301584_0,Chris Green,PRESENT,Non validated citizen science record,2018-09-23T10:04:00,2018,9,23,citizen science scuba dive observation,East China Sea,JP,Ariga Cable,"Ariga Cable, Tokashiki, Keramas, Okinawa",26.20660,127.344000,100.0,Google Maps,Chris Green,2018-09-23T00:00:00,http://eol.org/pages/1301584,incertae sedis,EOL,incertae sedis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KINGDOM,NaN,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:17.407Z,0.0,0.0,7.620,7.620,COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH...,True,False,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chelonioidea Baur 1893,DWC_ARCHIVE,2020-12-15T21:04:17.407Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1324,2247810893,diveboard:426434_1301584_0,CC_BY_NC_4_0,2019-05-11T13:12:27Z,https://www.diveboard.com/greencr83/D1FSaJ3,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:426434_1301584_0,diveboard:426434_1301584_0,Chris Green,PRESENT,Non validated citizen science record,2018-09-23T14:04:00,2018,9,23,citizen science scuba dive observation,East China Sea,JP,Sand Tra East,"Sand Tra East, Keramas, Okinawa",26.22570,127.359000,100.0,Google Maps,Chris Green,2018-09-23T00:00:00,http://eol.org/pages/1301584,incertae sedis,EOL,incertae sedis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KINGDOM,NaN,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:14.928Z,0.0,0.0,7.620,7.620,COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH...,True,False,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chelonioidea Baur 1893,DWC_ARCHIVE,2020-12-15T21:04:14.928Z,2020-12-11T15:02:29.629Z,True,JPN,Japan,JPN.32_1,Okinawa,JPN.32.31_1,Tokashiki,NaN,NaN
1331,2247810886,diveboard:426433_1301584_0,CC_BY_NC_4_0,2019-05-11T13:12:27Z,https://www.diveboard.com/greencr83/D1Bnvca,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:426433_1301584_0,diveboard:426433_1301584_0,Chris Green,PRESENT,Non validated citizen science record,2018-09-23T11:57:00,2018,9,23,citizen science scuba dive observation,East China Sea,JP,Gishippu Island,"Gishippu Island, Kerama Islands",26.23760,127.367000,100.0,Google Maps,Chris Green,2018-09-23T00:00:00,http://eol.org/pages/1301584,incertae sedis,EOL,incertae sedis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KINGDOM,NaN,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:18.272Z,0.0,0.0,6.860,6.860,COORDINATE_REPROJECTION_

In [79]:
df[df['issue'].str.contains('TAXON_MATCH_HIGHERRANK')]

,gbifID,identifier,license,modified,references,rights,rightsHolder,type,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,recordedBy,occurrenceStatus,occurrenceRemarks,eventDate,year,month,day,samplingProtocol,waterBody,countryCode,locality,verbatimLocality,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferenceSources,identifiedBy,dateIdentified,taxonID,scientificName,nameAccordingTo,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,issue,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,speciesKey,species,genericName,acceptedScientificName,verbatimScientificName,protocol,lastParsed,lastCrawled,repatriated,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
1150,2381502044,diveboard:434124_1156452_0,CC_BY_NC_4_0,2019-08-09T00:11:40Z,https://www.diveboard.com/jborgmann/D53Gq4z,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:434124_1156452_0,diveboard:434124_1156452_0,Jab,PRESENT,Non validated citizen science record,2019-08-06T02:15:00,2019,8,6,citizen science scuba dive observation,Caribbean Sea,JM,Ocho Rios,"Ocho Rios, Ocho Rios",18.41500,-77.07900,100.0,Google Maps,Jab,2019-08-06T00:00:00,http://eol.org/pages/1156452,"Dasyatis Rafinesque, 1810",EOL,Animalia,Chordata,Elasmobranchii,Myliobatiformes,Dasyatidae,Dasyatis,NaN,NaN,GENUS,ACCEPTED,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:18.330Z,0.0,0.0,6.095,6.095,COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH...,True,False,2419449,2419449.0,1,44.0,121.0,8214029.0,2195.0,2419449.0,NaN,NaN,Dasyatis,"Dasyatis Rafinesque, 1810","Dasyatis say (Lesueur, 1817)",DWC_ARCHIVE,2020-12-15T21:04:18.330Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2458,1946389924,diveboard:395565_21377949_0,CC_BY_NC_4_0,2018-04-17T19:04:13Z,https://www.diveboard.com/klipvis/D1MceX7,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:395565_21377949_0,diveboard:395565_21377949_0,Wilhelm,PRESENT,Non validated citizen science record,2017-04-29T11:55:00,2017,4,29,citizen science scuba dive observation,Gordons baai,ZA,Rocky Bay,"Rocky Bay, Gordons baai",-34.18890,18.81560,100.0,Google Maps,Wilhelm,2017-04-29T00:00:00,http://eol.org/pages/21377949,"Cadlina Bergh, 1878",EOL,Animalia,Mollusca,Gastropoda,Nudibranchia,Cadlinidae,Cadlina,NaN,NaN,GENUS,ACCEPTED,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:19.481Z,0.0,0.0,8.250,8.250,COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH...,True,False,2292396,2292396.0,1,52.0,225.0,980.0,4359821.0,2292396.0,NaN,NaN,Cadlina,"Cadlina Bergh, 1878",Cadlina sp. 2 South Africa,DWC_ARCHIVE,2020-12-15T21:04:19.481Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13072,1098922689,diveboard:228648_1156452_0,CC_BY_NC_4_0,2016-03-19T22:37:28Z,https://www.diveboard.com/michael-colligon/DMLxdr,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:228648_1156452_0,diveboard:228648_1156452_0,Michael Colligon,PRESENT,Non validated citizen science record,2015-05-30T11:33:00,2015,5,30,citizen science scuba dive observation,Atlantic Ocean,US,Aeolus,"Aeolus, Morehead City",34.26690,-76.63350,100.0,Google Maps,Michael Colligon,2015-05-30T00:00:00,http://eol.org/pages/1156452,"Dasyatis Rafinesque, 1810",EOL,Animalia,Chordata,Elasmobranchii,

In [81]:
df[df['issue'].str.contains('COUNTRY_DERIVED_FROM_COORDINATES')]['countryCode']

1572     BQ
6788     BQ
6796     SA
6944     VI
6961     VI
7072     VI
7094     VI
14945    VI
35689    CX
36585    GU
36590    GU
36595    GU
37625    VI
37629    VI
37630    VI
37634    VI
37639    VI
Name: countryCode, dtype: object

In [25]:
df['hasGeospatialIssues'].value_counts()

False    36437
True      1887
Name: hasGeospatialIssues, dtype: int64

In [26]:
df[df['hasGeospatialIssues'] == True]

,gbifID,identifier,license,modified,references,rights,rightsHolder,type,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,recordedBy,occurrenceStatus,occurrenceRemarks,eventDate,year,month,day,samplingProtocol,waterBody,countryCode,locality,verbatimLocality,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferenceSources,identifiedBy,dateIdentified,taxonID,scientificName,nameAccordingTo,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,issue,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,speciesKey,species,genericName,acceptedScientificName,verbatimScientificName,protocol,lastParsed,lastCrawled,repatriated,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
12,2993408634,diveboard:461013_356275_0,CC_BY_NC_4_0,2020-11-28T01:25:24Z,https://www.diveboard.com/epnfrnrush/D2KIvUP,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:461013_356275_0,diveboard:461013_356275_0,Epnfrnrush,PRESENT,Non validated citizen science record,2019-12-06T15:14:00,2019,12,6,citizen science scuba dive observation,Carribean Sea,ZZ,Hilma Hooker,"Hilma Hooker, Bonaire",12.1049,-68.2884,100.0,Google Maps,Epnfrnrush,2019-12-06T00:00:00,http://eol.org/pages/356275,"Myleus torquatus (Kner, 1858)",EOL,Animalia,Chordata,Actinopterygii,Characiformes,Serrasalmidae,Myloplus,torquatus,NaN,SPECIES,SYNONYM,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:19.334Z,0.0,0.0,12.190,12.190,COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COO...,True,True,2354468,7193779.0,1,44.0,204.0,537.0,4287106.0,2353403.0,7193779.0,Myloplus torquatus,Myleus,"Myloplus torquatus (Kner, 1858)","Myleus torquatus (Kner, 1858)",DWC_ARCHIVE,2020-12-15T21:04:19.334Z,2020-12-11T15:02:29.629Z,True,BES,"Bonaire, Sint Eustatius and Saba",BES.1_1,Bonaire,NaN,NaN,NaN,NaN
143,2993408503,diveboard:461376_585434_0,CC_BY_NC_4_0,2020-11-28T02:32:39Z,https://www.diveboard.com/epnfrnrush/D1ySW9P,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:461376_585434_0,diveboard:461376_585434_0,Epnfrnrush,PRESENT,Non validated citizen science record,2009-12-07T14:31:00,2009,12,7,citizen science scuba dive observation,Caribbean Sea,ZZ,Breezes House Reef,"Breezes House Reef, Curacao",12.0877,-68.9023,100.0,Google Maps,Epnfrnrush,2009-12-07T00:00:00,http://eol.org/pages/585434,"Aulostomus maculatus Valenciennes, 1841",EOL,Animalia,Chordata,Actinopterygii,Syngnathiformes,Aulostomidae,Aulostomus,maculatus,NaN,SPECIES,ACCEPTED,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:15.269Z,0.0,0.0,9.145,9.145,COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COO...,True,True,2332595,2332595.0,1,44.0,204.0,773.0,5903.0,2332594.0,2332595.0,Aulostomus maculatus,Aulostomus,"Aulostomus maculatus Valenciennes, 1841","Aulostomus maculatus Valenciennes, 1841",DWC_ARCHIVE,2020-12-15T21:04:15.269Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,2993408354,diveboard:461377_455703_0,CC_BY_NC_4_0,2020-11-28T02:32:39Z,https://www.diveboard.com/epnfrnrush/D227Aps,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:461377_455703_0,diveboard:461377_455703_0,Epnfrnrush,PRESENT,Non validated citizen science record,2009-12-08T15:47:00,2009,12,8,citizen science scuba dive observation,Caribbean Sea,ZZ,Breezes House Reef

In [27]:
df[df['hasGeospatialIssues'] == True]['issue'].value_counts()

COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;COUNTRY_INVALID;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE    1609
COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                     120
ZERO_COORDINATE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                                    115
ZERO_COORDINATE;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                         42
ZERO_COORDINATE;TAXON_MATCH_NONE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                     1
Name: issue, dtype: int64